In [1]:

# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


In [2]:

# Load your data
essays = pd.read_csv('train_essays.csv')
print(essays.head(10))  


         id  prompt_id                                               text  \
0  0059830c          0  Cars. Cars have been around since they became ...   
1  005db917          0  Transportation is a large necessity in most co...   
2  008f63e3          0  "America's love affair with it's vehicles seem...   
3  00940276          0  How often do you ride in a car? Do you drive a...   
4  00c39458          0  Cars are a wonderful thing. They are perhaps o...   
5  00da8c32          1  The electrol college system is an unfair syste...   
6  011dc2bc          1  Dear state senator, It is the utmost respect t...   
7  01448434          0  Fellow citizens, cars have become a major role...   
8  01c6e176          1  "It's official: The electoral college is unfai...   
9  0202ddf9          1  The Electoral College has been kept for centur...   

   generated  
0          0  
1          0  
2          0  
3          0  
4          0  
5          0  
6          0  
7          0  
8          0  
9 

In [3]:

def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Remove special characters
    text = re.sub(r'\W', ' ', text)
    
    # Remove numbers
    text = re.sub(r'\d', ' ', text)
    
    # Remove single characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    
    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    
    # Remove leading and trailing spaces
    text = text.strip()
    
    # Tokenize the text
    tokens = text.split()
    
    # Remove stopwords and stem the words
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens if word not in stopwords.words('english')]
    
    return ' '.join(tokens)


In [4]:

# Preprocess your data
# Apply the preprocessing function and create a new column
essays['cleaned_text'] = essays['text'].apply(preprocess_text)

# Print the number of preprocessed texts
print(f"Number of preprocessed texts: {len(essays['cleaned_text'])}")

# Print the first preprocessed text
print(f"First preprocessed text: {essays['cleaned_text'].iloc[0]}")


Number of preprocessed texts: 1378
First preprocessed text: car car around sinc becam famou henri ford creat built first modelt car play major role everi day live sinc peopl start question limit car usag would good thing limit use car might good thing like matter articl german suburb life goe without car elizabeth rosenth state automobil linchpin suburb middl class famili either shanghai chicago tend make home expert say huge impedi current effort reduc greenhous ga emiss tailpip passeng car respons percent greenhous ga emiss europ percent carintens area unit state car main reason greenhous ga emiss lot peopl drive around time get need go articl pari ban drive due smog robert duffer say pari day nearrecord pollut enforc partial drive ban clear air global citi also say monday motorist evennumb licens plate order leav car home fine euro fine order would appli oddnumb plate follow day car reason pollut entir citi like pari show bad car pollut caus entir citi likewis articl carfre day spin

In [5]:

# Extract features
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(essays['cleaned_text'])
y = essays['generated']


In [6]:

# Split your data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
# Train your model
model = LogisticRegression()
model.fit(X_train, y_train)



LogisticRegression()

In [8]:
# Make predictions on your test set
predictions = model.predict(X_test)
print(predictions)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [9]:
# Evaluate your model
print(f'Accuracy: {accuracy_score(y_test, predictions) * 100} %')

Accuracy: 99.63768115942028 %
